### 1. imports and configuration

In [3]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import os

# --- JUPYTER CONFIG ---
# Displays all columns (prevents ... truncation)
pd.set_option('display.max_columns', None) 

# --- PROJECT CONFIG ---
SYMBOL = "EURUSD"
START_YEAR = 2020
OUTPUT_FOLDER = "../data"  # Save data one level up
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

print(f"📂 Data will be saved to: {os.path.abspath(OUTPUT_FOLDER)}")

📂 Data will be saved to: C:\Users\casus\OneDrive\Desktop\AERO ORGANISED\MY WORK, PROJECTS ETC\My_projects\Project Autotrade\data


### Connection Function

In [4]:
def connect_to_mt5():
    """Attempts to initialize the MT5 connection."""
    if not mt5.initialize():
        print(f"❌ Initialize failed, error code: {mt5.last_error()}")
        return False
    
    # Check if we are connected to the correct account type
    account_info = mt5.account_info()
    if account_info:
        print(f"✅ Connected to: {account_info.server}")
        print(f"🔹 Account: {account_info.login} ({account_info.currency})")
        print(f"🔹 Balance: {account_info.balance}")
        return True
    return False

# Run connection test
connect_to_mt5()

✅ Connected to: FBS-Demo
🔹 Account: 105395845 (USD)
🔹 Balance: 20000.0


True

### The Data Fetcher

In [5]:
def fetch_data(symbol, timeframe, timeframe_name):
    """
    Fetches raw data from MT5 and returns a Pandas DataFrame.
    """
    # 1. Define Time Range (UTC)
    timezone = pytz.timezone("Etc/UTC")
    utc_from = datetime(START_YEAR, 1, 1, tzinfo=timezone)
    utc_to = datetime.now(timezone)
    
    print(f"\n📥 Fetching {symbol} [{timeframe_name}] from {START_YEAR} to Now...")
    
    # 2. Copy Rates
    rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
    
    # 3. Validation
    if rates is None or len(rates) == 0:
        print(f"❌ Error: No data received for {timeframe_name}.")
        print("   - Check if 'Max bars in chart' is set to Unlimited in MT5 options.")
        return None

    # 4. Convert to DataFrame
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    
    print(f"✅ Success! Retrieved {len(df):,} candles.")
    
    # Display the first 3 rows to verify data looks sane
    display(df.head(3))
    return df

### Execution & Saving

In [6]:
# --- 1. FETCH DAILY DATA (Supervisor) ---
df_daily = fetch_data(SYMBOL, mt5.TIMEFRAME_D1, "D1")

if df_daily is not None:
    path = f"{OUTPUT_FOLDER}/{SYMBOL}_D1.parquet"
    df_daily.to_parquet(path)
    print(f"💾 Saved Daily data to: {path}")

# --- 2. FETCH M15 DATA (Sniper) ---
df_m15 = fetch_data(SYMBOL, mt5.TIMEFRAME_M15, "M15")

if df_m15 is not None:
    path = f"{OUTPUT_FOLDER}/{SYMBOL}_M15.parquet"
    df_m15.to_parquet(path)
    print(f"💾 Saved M15 data to: {path}")

# --- 3. CLEANUP ---
mt5.shutdown()
print("\n🔌 Connection closed.")


📥 Fetching EURUSD [D1] from 2020 to Now...
✅ Success! Retrieved 1,552 candles.


,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2020-01-02,1.1212,1.12255,1.11640,1.11720,37991,20,0
2020-01-03,1.1172,1.11792,1.11241,1.11540,40083,21,0
2020-01-06,1.1163,1.12058,1.11565,1.11955,33765,21,0


💾 Saved Daily data to: ../data/EURUSD_D1.parquet

📥 Fetching EURUSD [M15] from 2020 to Now...
✅ Success! Retrieved 148,658 candles.


,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2020-01-02 00:00:00,1.12120,1.12168,1.12071,1.12112,76,56,0
2020-01-02 00:15:00,1.12077,1.12158,1.12077,1.12156,646,29,0
2020-01-02 00:30:00,1.12154,1.12161,1.12132,1.12158,85,30,0


💾 Saved M15 data to: ../data/EURUSD_M15.parquet

🔌 Connection closed.
